In [44]:
import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

In [45]:
def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""--------------------------------FUNCION PARA LIMPIAR NUMEROS A FLOAT--------------------------------------------------------"""

def Clean_column_number(column):
    column=column.apply(lambda x: float(str(x).strip().replace(",","")))
    return(column)

def default_column(default_columns,dataframe):     #Parametros (default_columns: Columnas predeterminadas,dataframe:)
    diccionay_default_column={}
    if len(default_columns)==len(dataframe.columns):
        for i in range(0,len(default_columns)):
            diccionay_default_column[dataframe.columns[i]]=default_columns[i]   
        dataframe=dataframe.rename(columns=diccionay_default_column)   #Remplazo las columnas con las de default
    else:
        None   #asumo columnas originales como estandar
    return(dataframe)

def Date_Null(dataframe,date):
    filter=(date!="00.00.0000")
    dataframe=dataframe.loc[filter]
    return(dataframe)

In [46]:
""" Search MB52 bulk"""
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][1]
password=Keys["Password"][1]
# Initialize session
Iniciar=SAP_GUI.SessionSAP(user,password)
# Start transsación
Transsacion="mb52"
provision="GRANELES"   
variant="AUXNVOSPT"
Buscar_MB52_GRANELES=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)


In [47]:
Name_GRANELES="GRANELES"  # Name file
SAP_GUI.Export_TXT2(Name_GRANELES,Iniciar)

In [48]:
Ruta_GRANELES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_GRANELES)+".txt"
data_GRANELES = pd.read_csv(Ruta_GRANELES, skiprows=1, delimiter='\t')

In [49]:
data_GRANELES=Clean_Columns(data_GRANELES)

In [50]:
default_column_MB52=['Material', 'Texto breve de material', 'Gpo.artíc.', 'Ce.', 'Alm.',
       'Lote', 'UMB', 'Libre utilización', 'En control calidad',
       'Valor libre util.', 'Valor en insp.cal.', 'Cad./FPC']   #Disposicion PROXI A VENCER


In [51]:
data_GRANELES=default_column(default_column_MB52,data_GRANELES) 

In [52]:
data_GRANELES=Date_Null(data_GRANELES,data_GRANELES['Cad./FPC'])

"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_GRANELES=data_GRANELES[data_GRANELES.Material.notnull()]
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].apply(lambda x: datetime.strptime(x,'%d.%m.%Y'))  #Change columns str to datetime

Data_Denominacion=Clean_Columns(pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Denominacion_Articulos.txt",delimiter='\t'))[["Grupo art.","Denom.gr.artíc."]]

data_GRANELES=pd.merge(Data_Denominacion,data_GRANELES,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


In [53]:
"""---------------------------------Filtro graneles vencidos-------------------"""
now=datetime.now()
month_vencido=now-timedelta(days=30) 
mask_vencido=(data_GRANELES["Cad./FPC"]<=now) & (data_GRANELES["Cad./FPC"]>=month_vencido)
data_GRANELES_vencidos=data_GRANELES.loc[mask_vencido]
#Sort values
data_GRANELES_vencidos=data_GRANELES_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_GRANELES_vencidos=data_GRANELES_vencidos.reset_index(drop=True)
data_GRANELES_vencidos["Cad./FPC"]=data_GRANELES_vencidos["Cad./FPC"].dt.date   #convet datetime to date 

In [54]:
# filter dates between now a 30 month later
now=datetime.now()
month=timedelta(days=6*30)   # Change limiter day(30 days= 1 month)
month=now+month
mask=(data_GRANELES["Cad./FPC"]>=now) & (data_GRANELES["Cad./FPC"]<=month)
data_GRANELES=data_GRANELES.loc[mask]
#Sort values
data_GRANELES=data_GRANELES.sort_values(by=["Cad./FPC","Libre utilización"])
data_GRANELES=data_GRANELES.reset_index(drop=True)
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].dt.date   #convet datetime to date 

In [55]:
data_GRANELES_vencidos

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Cad./FPC
0,C014,GR. Prebel,PB0020133,WIP VIOLET N°2 K7014-J,C014,1000.0,1001.0,10897141,KG,0,0,0,0,2023-06-21
1,C030,GR. Unilever,PB0093603,EGO GEL EXTREME MAX COL BET,C030,1000.0,1001.0,10901285,KG,0,0,0,0,2023-06-16
2,C295,GR.KOBA,PB0089717,CREMA EXF. CORP. AVENA - KOBA,C295,1000.0,1001.0,10883619,KG,0,0,0,0,2023-06-14
3,C295,GR.KOBA,PB0089716,CREMA EXF. CORP. POMGRANE - KOBA,C295,1000.0,1001.0,10895476,KG,0,0,0,0,2023-06-14
4,C136,GR.Biotecnick,PB0082173,SLN COLOR AMARILLO CIBA,C136,1000.0,1001.0,10886461,KG,0,0,0,0,2023-06-14
5,C038,GR. Avon,1-67645,AVON TRUE MICELLAR WATER RESTA,C038,1000.0,1001.0,10844582,KG,0,0,0,0,2023-06-14
6,C014,GR. Prebel,PB0083923,MF PAN STICK BASE 61 ORO VIEJO,C014,1000.0,1001.0,10900899,KG,0,0,0,0,2023-06-11
7,C014,GR. Prebel,PB0087764,AGUA MICELAR (NF),C014,1000.0,1001.0,10899981,KG,0,0,0,0,2023-06-09
8,C014,GR. Prebel,PB0077030,DEO PIES AEROSOL AFM ANTIBACTERIAL NF,C014,1000.0,1001.0,10900440,KG,0,0,0,0,2023-06-05
9,C014,GR. Prebel,PB0083919,MF MAJICAL COMPLEX,C014,1000.0,1001.0,10891518,KG,0,0,0,0,2023-06-02


In [56]:
"""------------DATAFRAME GRANALES PARA INFORME-----------------------------------------------------------------------"""

data_Graneles_informe=data_GRANELES[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]


data_Graneles_Vencidos_informe=data_GRANELES_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote', 'UMB',
       'Cad./FPC']]

data_Graneles_Vencidos_informe

data_Graneles_informe

,Material,Texto breve de material,Gpo.artíc.,Denom.gr.artíc.,Ce.,Alm.,Lote,Libre utilización,En control calidad,UMB,Valor libre util.,Valor en insp.cal.,Cad./FPC
0,PB0078131,GEL CAPILAR MÁXIMA FIJACIÓN (RW),C014,GR. Prebel,1000.0,1001.0,10899685,378.200,0,KG,"1,526,010",0,2023-06-22
1,PB0079251,FRAG. MALE MASS. AMBICIOSO NOVAV,C266,GR.NOVAVENTA FPT,1000.0,1001.0,10890449,20,0,KG,"832,875",0,2023-06-23
2,PB0060364,DEO-AP ROLLON JM FEMENINO FRAG. CLEAN,C220,GR.Jerónimo Martens,1000.0,1001.0,10899920,0,0,KG,0,0,2023-06-24
3,PB0093601,DEO-AP ROLL-ON ORIGINAL AFM,C014,GR. Prebel,1000.0,1001.0,10899908,0,0,KG,0,0,2023-06-24
4,PB0085121,LACA NORMAL NATURAL STYLING FULL,C241,GR.HENKEL,1000.0,1001.0,10902325,797,0,KG,"8,633,989",0,2023-06-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,PB0089408,50377954 CX2 AGUAS COLONIA VIOLETA,C042,GR. Natura,1000.0,1001.0,10901326,26,0,KG,"428,136",0,2023-12-06
62,PB0079457,50063072 SOL. COL. ALC. NAT. VERD. PUR.,C042,GR. Natura,1000.0,1001.0,10902110,0.970,0,KG,"6,726",0,2023-12-13
63,PB0023590,"50061043 SOL. COL. ACU. NAT. AMA5(0,05%)",C042,GR. Natura,1000.0,1001.0,10902316,0,"30,000",G,0,"322,198",2023-12-18
64,PB0023586,50063071 SOL. COL. ALC. NAT. NAR(0.1%),C042,GR. Natura,1000.0,1001.0,10902318,"10,000",0,G,"120,030",0,2023-12-18


In [57]:
data_Graneles_Vencidos_informe

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Cad./FPC
0,C014,GR. Prebel,PB0020133,WIP VIOLET N°2 K7014-J,C014,1000.0,1001.0,10897141,KG,2023-06-21
1,C030,GR. Unilever,PB0093603,EGO GEL EXTREME MAX COL BET,C030,1000.0,1001.0,10901285,KG,2023-06-16
2,C295,GR.KOBA,PB0089717,CREMA EXF. CORP. AVENA - KOBA,C295,1000.0,1001.0,10883619,KG,2023-06-14
3,C295,GR.KOBA,PB0089716,CREMA EXF. CORP. POMGRANE - KOBA,C295,1000.0,1001.0,10895476,KG,2023-06-14
4,C136,GR.Biotecnick,PB0082173,SLN COLOR AMARILLO CIBA,C136,1000.0,1001.0,10886461,KG,2023-06-14
5,C038,GR. Avon,1-67645,AVON TRUE MICELLAR WATER RESTA,C038,1000.0,1001.0,10844582,KG,2023-06-14
6,C014,GR. Prebel,PB0083923,MF PAN STICK BASE 61 ORO VIEJO,C014,1000.0,1001.0,10900899,KG,2023-06-11
7,C014,GR. Prebel,PB0087764,AGUA MICELAR (NF),C014,1000.0,1001.0,10899981,KG,2023-06-09
8,C014,GR. Prebel,PB0077030,DEO PIES AEROSOL AFM ANTIBACTERIAL NF,C014,1000.0,1001.0,10900440,KG,2023-06-05
9,C014,GR. Prebel,PB0083919,MF MAJICAL COMPLEX,C014,1000.0,1001.0,10891518,KG,2023-06-02


In [58]:
"""-----------------------------GENERO SUBTOTALES------------------------------------------------------------"""

data_GRANELES_Libre=Clean_column_number(data_GRANELES["Valor libre util."]).sum()
data_GRANELES_Calidad=Clean_column_number(data_GRANELES['Valor en insp.cal.']).sum()
Total_Graneles=data_GRANELES_Libre+data_GRANELES_Calidad
Total_Graneles="${:,.2f}".format(Total_Graneles)

In [59]:
Total_Graneles

'$97,193,355.00'

In [60]:
""" Search MB52 raw material"""

# Start transsación
Transsacion="mb52"
provision="MATERIA PRIMA"  
variant="AUXNVOSPT"
Buscar_MB52_MATERIAS=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)
# Save file



Name_Materias="MATERIAS"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_Materias,Iniciar)
# Pass the route and read file 
Ruta_Materias="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_Materias)+".txt"
data_Materias=pd.read_csv(Ruta_Materias,delimiter="\t")
# Change, drop and standardize columns
data_Materias=Clean_Columns(data_Materias)
data_Materias=default_column(default_column_MB52,data_Materias) 
data_Materias=Date_Null(data_Materias,data_Materias['Cad./FPC'])


"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_Materias=data_Materias[data_Materias.Material.notnull()]
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].apply(lambda x: datetime.strptime(str(x),'%d.%m.%Y'))  #Change columns str to datetime

data_Materias=pd.merge(Data_Denominacion,data_Materias,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


In [61]:
"""---------------------------Buscamos las clases de material------------- """
Data_Material_Total=data_Materias
Lista_tipo_Materias=data_Materias["Gpo.artíc."].drop_duplicates().reset_index(drop=True)


In [62]:
"""--------------------------------------MATERIAS PRIMAS VENCIDAS-------------------------"""

mask_vencido=(data_Materias["Cad./FPC"]<=now) & (data_Materias["Cad./FPC"]>=month_vencido)
data_Materias_vencidos=data_Materias.loc[mask_vencido]
#Sort values
data_Materias_vencidos=data_Materias_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_Materias_vencidos=data_Materias_vencidos.reset_index(drop=True)
data_Materias_vencidos["Cad./FPC"]=data_Materias_vencidos["Cad./FPC"].dt.date   #convet datetime to date 

In [63]:

mask=(data_Materias["Cad./FPC"]>=now) & (data_Materias["Cad./FPC"]<=month)
data_Materias=data_Materias.loc[mask]
data_Materias=data_Materias.sort_values(by=["Cad./FPC","Libre utilización"]).reset_index(drop=True)
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].dt.date

#Close Session
#SAP_GUI.Close_session(Iniciar)


"""------------DATAFRAME MATERIAS PARA INFORME-----------------------------------------------------------------------"""
data_Materia_informe=data_Materias[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]

data_Materia_informe_vencidos=data_Materias_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote', 'UMB',
       'Cad./FPC']]

In [64]:
"""-----------------------------GENERO SUBTOTALES------------------------------------------------------------"""

data_Materia_Libre=Clean_column_number(data_Materias["Valor libre util."]).sum()
data_Materia_Calidad=Clean_column_number(data_Materias['Valor en insp.cal.']).sum()
Total_Materia=data_Materia_Libre+data_Materia_Calidad
Total_Materia="${:,.2f}".format(Total_Materia)

In [65]:
Total_Materia

'$2,854,011,185.00'

In [66]:
data_Materias

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,En control calidad,Valor libre util.,Valor en insp.cal.,Cad./FPC
0,C026,MP. Unilever,U121460,CLORO METIL ISO TIAZOLINONA,C026,1000.0,1001.0,10891409.0,KG,1.088,0,0,0,2023-06-22
1,C041,MP. Natura,PB0077384,357686 AVOBENZONE - DSM,C041,1000.0,1001.0,10893581.0,KG,10.200,0,"924,333",0,2023-06-22
2,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10873278.0,G,74,0,0,0,2023-06-23
3,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10883672.0,G,87,0,0,0,2023-06-23
4,C242,MP.HENKEL,H2561691,EXTRA FINE GRANULATED SUGAR,C242,1000.0,1001.0,10888146.0,G,136,0,0,0,2023-06-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,C026,MP. Unilever,U28195,MONOESTEARATO DE GLICERILO,C026,1000.0,1001.0,10898733.0,KG,745.100,0,0,0,2023-12-16
452,C274,MP.BEIERSDORF,PB0085762,METHYLPARABEN/737,C274,1000.0,1001.0,10834883.0,KG,45.755,0,"1,491,051",0,2023-12-17
453,C041,MP. Natura,PB0090145,50373100 FRAG COCKTAILIFRAAR 264586,C041,1000.0,1001.0,10895153.0,KG,164.274,0,"24,212,345",0,2023-12-17
454,C329,MP.MILAGROS,PB0092451,ÁCIDO GLUTAMICO,C329,1000.0,1001.0,10884443.0,KG,85.300,0,"2,431,050",0,2023-12-18


In [67]:
"""Send email"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \
        .highlight_between(subset=["Cad./FPC"],color='#FF5733',left=now.date(),right=(now.date()+timedelta(days=7))) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>

    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Los graneles a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

html2="""
    <h2 style="text-align: center"> REPORTE CADUCIDAD DE MATERIAS PRIMAS</h2>
    <p> Por medio del presente informe se evidencia las materias primas segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>
    
    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Las materias primas proximas a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

Send = style_df(data_Graneles_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe)     #Style between LI and LS



# Definir el diccionario de formato
try:
    formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}
    formato_material= {'Ce.': '{:.0f}',"Alm.":'{:.0f}',"Lote":'{:.0f}'}
    # Aplicar el formato a la columna 'Altura'
    Send = Send.format(formato_Graneles)
    Send2=Send2.format(formato_material)
except:
    None

send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")
send_emails(Send2.to_html(),Total_Materia,emails=correos,htmlbody=html2,subject="REPORTE MATERIAS PRIMAS PROXIMOS A VENCER")

In [69]:
send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")

In [70]:
"""-------------------------------------Send email material y granel vencidos--------------------------------"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles y materias primas vencidas, para su correspondiente análisis. </p>

    <h2 style="color: black;" > GRANELES VENCIDOS</h4>

    <div"> {0} </div>

    <h2 style="color: black;" > MATERIA PRIMA VENCIDA </h4>
    

    <div"> {1} </div>


    <h4> Anticipo sinceros agradecimientos. </h4>
 """
formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}


Send = style_df(data_Graneles_Vencidos_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe_vencidos)     #Style between LI and LS
Send=Send.format(formato_Graneles)
Send2=Send2.format(formato_Graneles)
send_emails(Send.to_html(),Send2.to_html(),emails=correos,htmlbody=html,subject="REPORTE GRANELES Y MARIA PRIMA VENCIDO")
